In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.utils import to_categorical
import numpy as np
 
#Load Data with pandas, and parse the first column into datetime
# train=pd.read_csv('train.csv', parse_dates = ['Dates'])
# test=pd.read_csv('test.csv', parse_dates = ['Dates'])
data=pd.read_csv('data/combined_weather_crime_lunar_df_two.csv')
data.head()

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

,Unnamed: 0,DATE,CCN,STATION,NAME,DEWP,GUST,MAX,MIN,PRCP,SLP,SNDP,TEMP,WDSP,Year,Month,Date,Day,Illum
0,0,1/1/16,103,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",30.8,0.0,57.0,39.9,0.0,1020.6,0.0,44.0,8.0,2016,1,1,6,0.60
1,1,1/2/16,93,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",23.4,14.0,48.0,32.0,0.0,1020.2,0.0,39.5,6.4,2016,1,2,7,0.50
2,2,1/3/16,100,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",28.0,24.1,54.0,32.0,0.0,1013.9,0.0,40.9,5.1,2016,1,3,1,0.41
3,3,1/4/16,77,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.2,28.9,54.0,30.0,0.0,1017.8,0.0,35.6,12.5,2016,1,4,2,0.32
4,4,1/5/16,92,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.0,29.9,37.9,16.0,0.0,1034.3,0.0,24.2,12.7,2016,1,5,3,0.23


In [2]:
data.drop(data.columns[[0]], axis=1, inplace=True)
data.head()

,DATE,CCN,STATION,NAME,DEWP,GUST,MAX,MIN,PRCP,SLP,SNDP,TEMP,WDSP,Year,Month,Date,Day,Illum
0,1/1/16,103,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",30.8,0.0,57.0,39.9,0.0,1020.6,0.0,44.0,8.0,2016,1,1,6,0.60
1,1/2/16,93,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",23.4,14.0,48.0,32.0,0.0,1020.2,0.0,39.5,6.4,2016,1,2,7,0.50
2,1/3/16,100,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",28.0,24.1,54.0,32.0,0.0,1013.9,0.0,40.9,5.1,2016,1,3,1,0.41
3,1/4/16,77,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.2,28.9,54.0,30.0,0.0,1017.8,0.0,35.6,12.5,2016,1,4,2,0.32
4,1/5/16,92,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.0,29.9,37.9,16.0,0.0,1034.3,0.0,24.2,12.7,2016,1,5,3,0.23


In [3]:
data.rename(columns={"CCN": "CRIME-COUNT"}, inplace=True)

In [4]:
data['bins'] = np.nan

In [5]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,DEWP,GUST,MAX,MIN,PRCP,SLP,SNDP,TEMP,WDSP,Year,Month,Date,Day,Illum,bins
0,1/1/16,103,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",30.8,0.0,57.0,39.9,0.0,1020.6,0.0,44.0,8.0,2016,1,1,6,0.60,NaN
1,1/2/16,93,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",23.4,14.0,48.0,32.0,0.0,1020.2,0.0,39.5,6.4,2016,1,2,7,0.50,NaN
2,1/3/16,100,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",28.0,24.1,54.0,32.0,0.0,1013.9,0.0,40.9,5.1,2016,1,3,1,0.41,NaN
3,1/4/16,77,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.2,28.9,54.0,30.0,0.0,1017.8,0.0,35.6,12.5,2016,1,4,2,0.32,NaN
4,1/5/16,92,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.0,29.9,37.9,16.0,0.0,1034.3,0.0,24.2,12.7,2016,1,5,3,0.23,NaN


In [6]:
data.loc[data['CRIME-COUNT']<=75, ['bins']] = 'VeryLow'


In [7]:
data.loc[(data['CRIME-COUNT']>=76) & (data['CRIME-COUNT']<=90), ['bins']] = 'Low'

In [8]:
data.loc[(data['CRIME-COUNT']>=91) & (data['CRIME-COUNT']<=105), ['bins']] = 'Medium'

In [9]:
data.loc[(data['CRIME-COUNT']>=106) & (data['CRIME-COUNT']<=120), ['bins']] = 'High'

In [10]:
data.loc[(data['CRIME-COUNT']>=121, ['bins'])] = 'VeryHigh'

In [11]:
data['bins'].value_counts()

Medium      432
Low         396
High        223
VeryLow     188
VeryHigh     80
Name: bins, dtype: int64

In [12]:
data['CRIME-COUNT'].describe()

count    1319.000000
mean       93.359363
std        17.072853
min        28.000000
25%        82.000000
50%        93.000000
75%       104.000000
max       146.000000
Name: CRIME-COUNT, dtype: float64

In [13]:
data.isnull().any()

DATE           False
CRIME-COUNT    False
STATION        False
NAME           False
DEWP           False
GUST           False
MAX            False
MIN            False
PRCP           False
SLP            False
SNDP           False
TEMP           False
WDSP           False
Year           False
Month          False
Date           False
Day            False
Illum          False
bins           False
dtype: bool

In [14]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,DEWP,GUST,MAX,MIN,PRCP,SLP,SNDP,TEMP,WDSP,Year,Month,Date,Day,Illum,bins
0,1/1/16,103,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",30.8,0.0,57.0,39.9,0.0,1020.6,0.0,44.0,8.0,2016,1,1,6,0.60,Medium
1,1/2/16,93,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",23.4,14.0,48.0,32.0,0.0,1020.2,0.0,39.5,6.4,2016,1,2,7,0.50,Medium
2,1/3/16,100,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",28.0,24.1,54.0,32.0,0.0,1013.9,0.0,40.9,5.1,2016,1,3,1,0.41,Medium
3,1/4/16,77,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.2,28.9,54.0,30.0,0.0,1017.8,0.0,35.6,12.5,2016,1,4,2,0.32,Low
4,1/5/16,92,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.0,29.9,37.9,16.0,0.0,1034.3,0.0,24.2,12.7,2016,1,5,3,0.23,Medium


In [15]:
X = data.drop(["bins", "DATE", "STATION", "NAME", "CRIME-COUNT", "Year"], axis=1)
y = data["bins"]

In [16]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,DEWP,GUST,MAX,MIN,PRCP,SLP,SNDP,TEMP,WDSP,Year,Month,Date,Day,Illum,bins
0,1/1/16,103,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",30.8,0.0,57.0,39.9,0.0,1020.6,0.0,44.0,8.0,2016,1,1,6,0.60,Medium
1,1/2/16,93,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",23.4,14.0,48.0,32.0,0.0,1020.2,0.0,39.5,6.4,2016,1,2,7,0.50,Medium
2,1/3/16,100,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",28.0,24.1,54.0,32.0,0.0,1013.9,0.0,40.9,5.1,2016,1,3,1,0.41,Medium
3,1/4/16,77,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.2,28.9,54.0,30.0,0.0,1017.8,0.0,35.6,12.5,2016,1,4,2,0.32,Low
4,1/5/16,92,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.0,29.9,37.9,16.0,0.0,1034.3,0.0,24.2,12.7,2016,1,5,3,0.23,Medium


In [17]:
# X['TMAX'] = X['TMAX'].astype(float)
# X['TMIN'] = X['TMIN'].astype(float)
# X['Month'] = X['Month'].astype(float)
# X['Date'] = X['Date'].astype(float)
# X['Day'] = X['Illum'].astype(float)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [21]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [23]:
model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=13))
model.add(Dense(units=400, activation='sigmoid'))
# model.add(Dense(units=300, activation='sigmoid'))
# model.add(Dense(units=100, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(units=5, activation='softmax'))

W0817 15:39:06.367079 4669162944 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [24]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               2800      
_________________________________________________________________
dense_1 (Dense)              (None, 400)               80400     
_________________________________________________________________
dropout (Dropout)            (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 2005      
Total params: 85,205
Trainable params: 85,205
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Epoch 1/1000
989/989 - 0s - loss: 1.6624 - acc: 0.2963
Epoch 2/1000
989/989 - 0s - loss: 1.5878 - acc: 0.3165
Epoch 3/1000
989/989 - 0s - loss: 1.5599 - acc: 0.3205
Epoch 4/1000
989/989 - 0s - loss: 1.4952 - acc: 0.3367
Epoch 5/1000
989/989 - 0s - loss: 1.4699 - acc: 0.3509
Epoch 6/1000
989/989 - 0s - loss: 1.4678 - acc: 0.3539
Epoch 7/1000
989/989 - 0s - loss: 1.4263 - acc: 0.3347
Epoch 8/1000
989/989 - 0s - loss: 1.4175 - acc: 0.3418
Epoch 9/1000
989/989 - 0s - loss: 1.4098 - acc: 0.3711
Epoch 10/1000
989/989 - 0s - loss: 1.4013 - acc: 0.3610
Epoch 11/1000
989/989 - 0s - loss: 1.3843 - acc: 0.3741
Epoch 12/1000
989/989 - 0s - loss: 1.3764 - acc: 0.3610
Epoch 13/1000
989/989 - 0s - loss: 1.3886 - acc: 0.3822
Epoch 14/1000
989/989 - 0s - loss: 1.3609 - acc: 0.3964
Epoch 15/1000
989/989 - 0s - loss: 1.3725 - acc: 0.3903
Epoch 16/1000
989/989 - 0s - loss: 1.3593 - acc: 0.3640
Epoch 17/1000
989/989 - 0s - loss: 1.3577 - acc: 0.3822
Epoch 18/1000
989/989 - 0s - loss: 1.3569 - acc: 0.3802
E

Epoch 147/1000
989/989 - 0s - loss: 1.2185 - acc: 0.4540
Epoch 148/1000
989/989 - 0s - loss: 1.2226 - acc: 0.4580
Epoch 149/1000
989/989 - 0s - loss: 1.2130 - acc: 0.4580
Epoch 150/1000
989/989 - 0s - loss: 1.2135 - acc: 0.4439
Epoch 151/1000
989/989 - 0s - loss: 1.2153 - acc: 0.4540
Epoch 152/1000
989/989 - 0s - loss: 1.2119 - acc: 0.4742
Epoch 153/1000
989/989 - 0s - loss: 1.2084 - acc: 0.4601
Epoch 154/1000
989/989 - 0s - loss: 1.2041 - acc: 0.4651
Epoch 155/1000
989/989 - 0s - loss: 1.2086 - acc: 0.4621
Epoch 156/1000
989/989 - 0s - loss: 1.2037 - acc: 0.4601
Epoch 157/1000
989/989 - 0s - loss: 1.2049 - acc: 0.4641
Epoch 158/1000
989/989 - 0s - loss: 1.2039 - acc: 0.4803
Epoch 159/1000
989/989 - 0s - loss: 1.2065 - acc: 0.4550
Epoch 160/1000
989/989 - 0s - loss: 1.2263 - acc: 0.4530
Epoch 161/1000
989/989 - 0s - loss: 1.2084 - acc: 0.4520
Epoch 162/1000
989/989 - 0s - loss: 1.2100 - acc: 0.4621
Epoch 163/1000
989/989 - 0s - loss: 1.1897 - acc: 0.4631
Epoch 164/1000
989/989 - 0s - l

Epoch 291/1000
989/989 - 0s - loss: 1.1146 - acc: 0.5116
Epoch 292/1000
989/989 - 0s - loss: 1.1284 - acc: 0.5035
Epoch 293/1000
989/989 - 0s - loss: 1.1103 - acc: 0.5035
Epoch 294/1000
989/989 - 0s - loss: 1.1098 - acc: 0.5096
Epoch 295/1000
989/989 - 0s - loss: 1.1062 - acc: 0.5025
Epoch 296/1000
989/989 - 0s - loss: 1.1097 - acc: 0.4954
Epoch 297/1000
989/989 - 0s - loss: 1.1082 - acc: 0.5147
Epoch 298/1000
989/989 - 0s - loss: 1.1155 - acc: 0.5207
Epoch 299/1000
989/989 - 0s - loss: 1.1132 - acc: 0.5066
Epoch 300/1000
989/989 - 0s - loss: 1.1178 - acc: 0.5258
Epoch 301/1000
989/989 - 0s - loss: 1.0866 - acc: 0.5116
Epoch 302/1000
989/989 - 0s - loss: 1.0912 - acc: 0.5308
Epoch 303/1000
989/989 - 0s - loss: 1.1091 - acc: 0.5126
Epoch 304/1000
989/989 - 0s - loss: 1.1106 - acc: 0.5086
Epoch 305/1000
989/989 - 0s - loss: 1.1069 - acc: 0.5167
Epoch 306/1000
989/989 - 0s - loss: 1.1057 - acc: 0.5076
Epoch 307/1000
989/989 - 0s - loss: 1.0899 - acc: 0.5379
Epoch 308/1000
989/989 - 0s - l

Epoch 435/1000
989/989 - 0s - loss: 1.0302 - acc: 0.5480
Epoch 436/1000
989/989 - 0s - loss: 1.0071 - acc: 0.5571
Epoch 437/1000
989/989 - 0s - loss: 1.0145 - acc: 0.5501
Epoch 438/1000
989/989 - 0s - loss: 1.0282 - acc: 0.5521
Epoch 439/1000
989/989 - 0s - loss: 1.0155 - acc: 0.5642
Epoch 440/1000
989/989 - 0s - loss: 1.0029 - acc: 0.5733
Epoch 441/1000
989/989 - 0s - loss: 1.0133 - acc: 0.5662
Epoch 442/1000
989/989 - 0s - loss: 1.0196 - acc: 0.5602
Epoch 443/1000
989/989 - 0s - loss: 1.0043 - acc: 0.5551
Epoch 444/1000
989/989 - 0s - loss: 1.0166 - acc: 0.5592
Epoch 445/1000
989/989 - 0s - loss: 1.0032 - acc: 0.5693
Epoch 446/1000
989/989 - 0s - loss: 1.0255 - acc: 0.5501
Epoch 447/1000
989/989 - 0s - loss: 0.9980 - acc: 0.5885
Epoch 448/1000
989/989 - 0s - loss: 1.0013 - acc: 0.5662
Epoch 449/1000
989/989 - 0s - loss: 0.9962 - acc: 0.5672
Epoch 450/1000
989/989 - 0s - loss: 1.0089 - acc: 0.5652
Epoch 451/1000
989/989 - 0s - loss: 0.9962 - acc: 0.5662
Epoch 452/1000
989/989 - 0s - l

Epoch 579/1000
989/989 - 0s - loss: 0.9165 - acc: 0.6127
Epoch 580/1000
989/989 - 0s - loss: 0.9061 - acc: 0.6279
Epoch 581/1000
989/989 - 0s - loss: 0.9024 - acc: 0.6067
Epoch 582/1000
989/989 - 0s - loss: 0.9166 - acc: 0.6097
Epoch 583/1000
989/989 - 0s - loss: 0.9099 - acc: 0.6218
Epoch 584/1000
989/989 - 0s - loss: 0.9108 - acc: 0.6026
Epoch 585/1000
989/989 - 0s - loss: 0.8814 - acc: 0.6249
Epoch 586/1000
989/989 - 0s - loss: 0.8957 - acc: 0.6138
Epoch 587/1000
989/989 - 0s - loss: 0.9061 - acc: 0.6360
Epoch 588/1000
989/989 - 0s - loss: 0.8983 - acc: 0.6229
Epoch 589/1000
989/989 - 0s - loss: 0.8948 - acc: 0.6127
Epoch 590/1000
989/989 - 0s - loss: 0.9066 - acc: 0.6208
Epoch 591/1000
989/989 - 0s - loss: 0.8947 - acc: 0.6107
Epoch 592/1000
989/989 - 0s - loss: 0.8890 - acc: 0.5986
Epoch 593/1000
989/989 - 0s - loss: 0.9032 - acc: 0.6148
Epoch 594/1000
989/989 - 0s - loss: 0.8985 - acc: 0.6047
Epoch 595/1000
989/989 - 0s - loss: 0.8916 - acc: 0.6138
Epoch 596/1000
989/989 - 0s - l

Epoch 723/1000
989/989 - 0s - loss: 0.7824 - acc: 0.6562
Epoch 724/1000
989/989 - 0s - loss: 0.8011 - acc: 0.6532
Epoch 725/1000
989/989 - 0s - loss: 0.8001 - acc: 0.6562
Epoch 726/1000
989/989 - 0s - loss: 0.8182 - acc: 0.6633
Epoch 727/1000
989/989 - 0s - loss: 0.8125 - acc: 0.6542
Epoch 728/1000
989/989 - 0s - loss: 0.8101 - acc: 0.6542
Epoch 729/1000
989/989 - 0s - loss: 0.8106 - acc: 0.6522
Epoch 730/1000
989/989 - 0s - loss: 0.7821 - acc: 0.6775
Epoch 731/1000
989/989 - 0s - loss: 0.7992 - acc: 0.6532
Epoch 732/1000
989/989 - 0s - loss: 0.8197 - acc: 0.6512
Epoch 733/1000
989/989 - 0s - loss: 0.8079 - acc: 0.6441
Epoch 734/1000
989/989 - 0s - loss: 0.7786 - acc: 0.6663
Epoch 735/1000
989/989 - 0s - loss: 0.7934 - acc: 0.6603
Epoch 736/1000
989/989 - 0s - loss: 0.8051 - acc: 0.6754
Epoch 737/1000
989/989 - 0s - loss: 0.7948 - acc: 0.6684
Epoch 738/1000
989/989 - 0s - loss: 0.7879 - acc: 0.6552
Epoch 739/1000
989/989 - 0s - loss: 0.7947 - acc: 0.6694
Epoch 740/1000
989/989 - 0s - l

In [ ]:
# X_train_scaled

In [ ]:
# y_train_categorical.shape

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")